In [ ]:
import numpy as np
# наборы данных для экспериментов
from tensorflow.keras.datasets import cifar10
# последовательная модель (стек слоев)
from tensorflow.keras.models import Sequential, Model
# полносвязный слой и слой выпрямляющий матрицу в вектор
from tensorflow.keras.layers import Dense, Flatten, Input
# слой выключения нейронов и слой нормализации выходных данных (нормализует данные в пределах текущей выборки)
from tensorflow.keras.layers import Dropout, BatchNormalization, SpatialDropout2D, GaussianDropout
# слои свертки и подвыборки
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
# работа с обратной связью от обучающейся нейронной сети
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# вспомогательные инструменты
from tensorflow.keras import utils
from tensorflow.keras.regularizers import *

import os
from tensorflow.random import set_seed
def seed_everything(seed):
    np.random.seed(seed)
    set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
seed_everything(seed)

# работа с изображениями
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
# Размер мини-выборки
batch_size = 64
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 40
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

### Загрузка данных

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Нормализация данных

In [ ]:
X_train = X_train.reshape((50000, img_cols, img_rows, img_channels))
X_train = X_train.astype('float32')
X_test = X_test.reshape((10000, img_cols, img_rows, img_channels))
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Преобразование в One Hot Encoding

In [ ]:
Y_train = utils.to_categorical(y_train, nb_classes)
Y_test = utils.to_categorical(y_test, nb_classes)

### Инициализация модели нейросети

In [ ]:
from tensorflow.keras import layers, Model
def conv2D_block(x, num_conv=2, num_filters=32, drop_rate=0.25):
    for _ in range(num_conv):
        x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)
    return x

def conv2D_net(x, nb_classes, num_block=3, num_conv_strategy=[1,2,2],
               num_filters_st=[32, 64, 128], drop_rates=[0.25, 0.25, 0.25]):
    x = Conv2D(16, (3, 3), activation='relu', 
               padding='same',input_shape=(img_rows, img_cols, img_channels))(x)
    for i in range(num_block):
        x = conv2D_block(x, num_conv_strategy[i], num_filters_st[i], drop_rates[i])
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.75)(x)
    x = Dense(nb_classes, activation='softmax')(x)
    return x

input_layer = Input(shape=(img_rows, img_cols, img_channels), name='input_layer')
output_layer = conv2D_net(input_layer, nb_classes)
model = Model(input_layer, output_layer)


### Сведения об инициализированной модели

In [ ]:
model.summary()

### Компиляция модели

In [ ]:
callbacks_list = [EarlyStopping(monitor='val_loss', patience=5),
                  ModelCheckpoint(filepath='my_model.h5',
                                  monitor='val_loss',
                                  save_best_only=True),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
                  ] 
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

### Обучение

In [ ]:
%%time
history = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              callbacks=callbacks_list,
              validation_split=0.1,
              verbose=1)

### Оценка качества обучения

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1)
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))

In [ ]:
plt.plot(history.history['accuracy'], 
         label='Доля правильных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля правильных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля правильных ответов')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], 
         label='Оценка потерь на обучающем наборе')
plt.plot(history.history['val_loss'], 
         label='Оценка потерь на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Оценка потерь')
plt.legend()
plt.show()